# ***Clustering sobre noticias del periodico: El Tiempo (COL)***

Deseamos saber los temas de las noticias, asi como un resumen de estos en busca de una actualizacion via este periodo Colombiano. Para esto vamos a seguir estos pasos:

1. Obtencion de urls de las noticias.
2. Obtencion del texto de la noticia.
3. Generacion de embeddings de las noticias con [`Ollama`](https://ollama.com/) y la libreria [`langchain`](https://python.langchain.com/docs/introduction/).
4. Metodos de clustering.
5. Uso de LLM local (en Colab con Ollama) para obtener resumenes, insight y caractetizaciones de los cluster formados.
6. Discusion de resultados.


In [1]:
import requests
import pandas as pd
from io import StringIO

## ***Obtener urls de noticias del Tiempo***

In [ ]:
url_new = (
    """https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar"""
    """-el-titulo-en-la-proxima-carrera-en-japon-3385519"""
)

session = requests.Session()
# Peticion HTTP
response = session.get(url_new)
# Respuesta peticion
print("Respuesta a la peticion url:", response.status_code)
print(f"{100 * '='}")
# Contenido de la pagina web
print(response.text)

In [ ]:
url = 'https://www.eltiempo.com/sitemap-articles-current.xml'
session = requests.Session()
response = session.get(url)
if response.raise_for_status:
    print(f"OK, with the page: {url}. Status Code: {response.status_code}\n")
    print(f"Text from Page:")
    print(response.text[: 1000])
else:
    print(f"Problem in the page: {url}")

In [ ]:
namespaces = {
    "ns": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "news": "http://www.google.com/schemas/sitemap-news/0.9",
    "video": "http://www.google.com/schemas/sitemap-video/1.1"
}

# Hacemos lectura del texto XML
df_urls_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//ns:url", namespaces=namespaces)
df_urls_news_el_tiempo

In [ ]:
# Hacemos lectura del texto XML
df_title_date_keywords_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//news:news", namespaces=namespaces)
df_title_date_keywords_news_el_tiempo

In [ ]:
df_urls_news_el_tiempo = (
    df_urls_news_el_tiempo[["loc"]]
    .merge(
        df_title_date_keywords_news_el_tiempo.drop(columns=["publication"]),
        left_index=True,
        right_index=True        
    )
    .rename(columns={"loc": "url_page"})
)

df_urls_news_el_tiempo

## ***Obtener el texto de las noticias***

In [7]:
from bs4 import BeautifulSoup

In [ ]:
# Ejemplo de url a visitar:
df_urls_news_el_tiempo.iloc[0].loc["url_page"]

In [ ]:
%%time
# Primeras 20 urls de noticias.
list_urls = df_urls_news_el_tiempo["url_page"].head(5).to_list()

for url in list_urls:
    print("Trabajando en la URL:", url)
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html.parser")
    
    # Verificar si el request es exitosa
    if request.raise_for_status:
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])     
        print(f"Autor: {author_tag}")
        print(f"Contenido noticia: {content}")
        print()

## ***Paralelizar la tarea de obtencion del contenido de las noticias***

In [ ]:
def get_content_news_from_url(url: str) -> dict:
    """
    Obtiene el contenido de una noticia desde una URL, extrayendo el autor y el cuerpo de la noticia.

    :param url: URL de la página de la noticia.
    :return: Diccionario con la URL, autor y contenido de la noticia.
    :raises ValueError: Si hay un problema con el status_code de la solicitud.
    """
    session = requests.Session()
    request = session.get(url)
    
    if request.raise_for_status:
        soup = BeautifulSoup(request.content, "html.parser")
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])
        
        return {
            "url_page": url,
            "autor": author_tag,
            "news_content": content            
        }
    else:
        ValueError(
            f"Problemas con el status_code: {request.status_code}"
        )
        

get_content_news_from_url(df_urls_news_el_tiempo.iloc[0].loc["url_page"]) 

In [ ]:
import os
from typing import Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

num_cores = os.cpu_count()
print(f"Número de cores disponibles: {num_cores}")

# Numero de cores a usar.
num_cores = num_cores - 2

def process_url(url: str) -> Dict:
    return get_content_news_from_url(url)


def parallel_process_urls(urls: list[str]) -> list[Dict]:
    """
    Procesa múltiples URLs en paralelo utilizando ThreadPoolExecutor,
    dejando 2 cores libres.
    
    :param urls: Lista de URLs a procesar.
    :return: Lista de diccionarios con la información extraída de cada URL.
    """
    results = []
    
    with ThreadPoolExecutor(max_workers=num_cores) as executor:
        futures = [executor.submit(process_url, url) for url in urls]
        
        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error procesando la URL: {e}")
    
    return results

# Lista de URLs a procesar.
urls_to_process = df_urls_news_el_tiempo["url_page"].tolist()
# Procesamos todas las URLs en paralelo
processed_results = parallel_process_urls(urls_to_process)
# Resultados
print(processed_results)
pd.DataFrame(processed_results)

In [ ]:
df_news_el_tiempo = df_urls_news_el_tiempo.merge(
    pd.DataFrame(processed_results),
    on=["url_page"]    
)

display(df_news_el_tiempo)
df_news_el_tiempo.to_parquet("data/df_noticias_el_tiempo.parquet")
pd.read_parquet("data/df_noticias_el_tiempo.parquet")

## ***Usar un LLM local***

In [ ]:
# Instalar Ollama (disponible para Windows, Linux y MacOS)
# Servir (ollama serve)
# Descargar los modelos:
# 1. llama3.2:3b-instruct-q8_0 --> (ollama pull llama3.2:3b-instruct-q8_0)
# 2. llama3.1:8b --> (ollama pull llama3.1:8b)
# 3. qwen2.5:7b-instruct-q8_0 --> (ollama pull qwen2.5:7b-instruct-q8_0)

In [ ]:
# Uso es colab

In [ ]:
# Generar un chat con un llm_ollama
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(
    model="llama3.2:3b-instruct-q8_0",
    temperature=0.1,
    num_predict=1024,
    verbose=True
)

llm_response = llm_ollama.invoke("¿Cuál es la segunda letra del alfabeto griego?")
print(llm_response)

In [ ]:
%%time
llm_ollama.invoke(
    """Dame un lista de 20 personajes influyentes en la actualidad"""
    """mundial. No olvides incluir diferentes angulos de la vida. """
    """Solo debes darme la lista sin decir nada mas. Por ejemplo si """
    """decides incluir a Pepito Perez (suponiendo que es influyente """
    """actualmente) respuesta debe ser: Pepito Perez"""
    )

In [ ]:
# PromptTemplate y respuesta como string
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida en diferentes aspectos. Responde usando solo JSON con """
            """llave el nombre del personaje y valor su biografia, """
            """Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))"""),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

print(prompt)

chain = prompt | llm_ollama | StrOutputParser()
chain.invoke(
    {
        "user_input": "Barack Obama"
    }
)

In [ ]:
# Usamos JSON-format para parsear la respuesta del LLM
from langchain_core.output_parsers import JsonOutputParser

json_llm = ChatOllama(
    model="llama3.2:3b-instruct-q8_0",
    temperature=0.1,
    num_predict=-1,
    format="json"
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida ampliamente en diferentes aspectos. No debes escatimar """
            """en la longitud del texto de resumen. Responde usando solo JSON """
            """format con llave el nombre del personaje y valor su biografia"""
            """Por ejemplo: dict(Pepito Perez="Pepito Perez ... (biografia)","""
            """ Benito Camelas="Benito Camelas ... (biografia)"). Debes """
            """seguir al pie de la letra el formato"""
            ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

chain = prompt | json_llm | JsonOutputParser()
llm_answer = chain.invoke({"user_input": "Dame la biografia de 2 personajes"})
print(llm_answer, type(llm_answer))

### ***Embeddings***

In [ ]:
%%time
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="llama3.1:8b",
)

question = ["¿Cuál es la segunda letra del alfabeto griego?"]
embed_question = ollama_embeddings.embed_documents(question)
print("Embeddings:")
pd.DataFrame(embed_question, index=question)

In [ ]:
# Ejemplo: (Relevancia - Distancia)
from sklearn.metrics.pairwise import cosine_similarity
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="llama3.1:8b",
)

texts = [
    "Alpha es la primera letra del alfabeto griego",
    "A es la primera letra del alfabeto latino",
    "Beta es la segunda letra del alfabeto griego",
    "B es la segunda letra del alfabeto latino"
]
query = ["¿Cuál es la segunda letra del alfabeto griego?"]
embeds = ollama_embeddings.embed_documents(texts)
embed_query = ollama_embeddings.embed_documents(query)
print(f"Long Ollama Embeddings: {len(embed_query[0])}")
print(f"Cosine Similarities: \n {cosine_similarity(embeds, embed_query)}")

pd.DataFrame(
    cosine_similarity(embeds, embed_query),
    index=texts,
    columns=query
).sort_values(by=query, ascending=False)

In [ ]:
# Embedding de las noticias.
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="llama3.1:8b",
)

df_temp = df_news_el_tiempo.sample(20)
embed_news = ollama_embeddings.embed_documents(df_temp["news_content"].to_list())
print("Embeddings:")
pd.DataFrame(embed_news, index=df_temp["url_page"])

## ***K-means sobre los embeddings***

- Esta parte esta en construccion.

In [20]:
# k-means sobre los embeddings
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
scaled_data = StandardScaler().fit_transform(embed_news)
kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(scaled_data)
df_temp["cluster"] = kmeans.labels_
display(df_temp["cluster"].value_counts())
df_temp

In [22]:
# Seleccionamos un modelo "mas" inteligente.
llm_ollama = ChatOllama(
    model="qwen2.5:7b-instruct-q8_0",
    temperature=0.1,
    num_predict=-1,
    num_ctx=32000
)

prompt_caracterizacion = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que identica las tematicas generales de """
            """las noticias que te van a proporcionar pues estas noticias se """
            """supone estas relacionadas por sus embeddings. La respuesta de los """
            """temas deben ser similitudes precisamente semanticas, debes dar """
            """tu respuesta con el titulo que determina porque son similares estas """
            """noticias ademas de 5 bullets. Finalmente, daras un resumen al estilo """
            """de historia de todas las noticias para poner al dia en noticias """
            """ de la tematicas que las hace coincidentes al usuario. Un resumen"""
            """ por cada noticia."""
            """Debes decirme cuantas noticias te fueron proporcionadas"""         
            ),
        HumanMessagePromptTemplate.from_template("{news}")
    ]
)

llm_chain = prompt_caracterizacion | llm_ollama | StrOutputParser()

In [ ]:
import json
from pprint import pprint

list_news = df_temp.query("cluster == 0")["news_content"].to_list()
dict_news_content = {f"noticia_{i+1}": list_news[i] for i in range(len(list_news))}
pprint(dict_news_content)

In [ ]:
result = json.dumps(dict_news_content)
response_cluster_0 = llm_chain.invoke({"news": result})
print(response_cluster_0)